In [2]:
import pandas as pd
import numpy as np

In [3]:
test_file = 'symbol_prediction_test.csv'

In [4]:
test_df = pd.read_csv(test_file, sep=',', engine='c')

In [5]:
test_df.head()

,id,text
0,1,"""Take care, Lenox,"" exclaimed Zaidie, taking a..."
1,2,We followed him. When the three of us were bac...
2,3,"Ten years ago Lavalle, ""that imperturbable dre..."
3,4,"""I think I'd rather se■ what the rings are lik..."
4,5,"Surprisingly, she did shut up. Gordon went to ..."


In [75]:
test_df['text'][33]

'"Larr■," I said gently, "I was there."'

In [6]:
missing_symbol = '■'

In [7]:
from gensim.models import Word2Vec

In [10]:
import hunspell
spellchecker = hunspell.HunSpell('hunspell/en_GB-large.dic', 'hunspell/en_GB-large.aff')

In [70]:
spellchecker.suggest('Larr■')[0]

'Larry'

In [108]:
import re

In [54]:
print(spellchecker.suggest('se■'))
print(test_df['text'][3].find(missing_symbol))
print(test_df['text'][3].find(' ', 22))
print(test_df['text'][3].rfind(' ', 0, 22))
print(test_df['text'][3][19:24].strip())
# print(test_df['text'][3][19:24].strip(), ':', spellchecker.spell(test_df['text'][3][19:24].strip()))
print(test_df['text'][3][19:22].strip(), ':', spellchecker.spell(test_df['text'][3][19:22].strip()))
print(test_df['text'][3][23:24], ':', spellchecker.spell(test_df['text'][3][23:24]))

['see', 'sea', 'sen', 'ser', 'set', 'sec', 'sem', 'sew']
22
23
19
se■
se : False
  : True


In [125]:
space_symbol = ' '
empty_symbol = ''
separators = [' ', '!', '?', '"', '\'', '.', ',', ':', ';', '_', '-']
indices = []
predicted_symbols = []
for index, line in enumerate(test_df['text']):
    indices.append(index+1)
    index_of_missing = line.find(missing_symbol)
    index_of_prev_space = line.rfind(space_symbol, 0, index_of_missing)
    index_of_next_space = line.find(space_symbol, index_of_missing+1)
    prev_word = line[index_of_prev_space:index_of_missing].strip(''.join(separators))
    next_word = line[index_of_missing+1:index_of_next_space+1].strip(''.join(separators))
    # if missing_symbol is a delimiter - store ' ' as a prediction
    if (prev_word and spellchecker.spell(prev_word)) and not next_word:
        if index_of_missing+1 >= len(line):
            predicted_symbols.append('.')
            continue
        if line[index_of_missing+1] in separators:
            predicted_symbols.append(empty_symbol)
        else:
            predicted_symbols.append(space_symbol)
        continue

    if (next_word and spellchecker.spell(next_word)) and not prev_word:
        if index_of_missing-1 < 0:
            predicted_symbols.append(empty_symbol)
            continue
        if line[index_of_missing-1] in separators:
            predicted_symbols.append(empty_symbol)
        else:
            predicted_symbols.append(space_symbol)
        continue

    if (prev_word and spellchecker.spell(prev_word)) and (next_word and spellchecker.spell(next_word)):
        predicted_symbols.append(space_symbol)
        continue

    word_with_missing_symbol = line[index_of_prev_space+1:index_of_next_space].strip(''.join(separators))
    index_of_missing = word_with_missing_symbol.find(missing_symbol)
    if index_of_missing == -1:
#         print('Error at index:', index)
        predicted_symbols.append(space_symbol)
    else:
        all_suggested = spellchecker.suggest(word_with_missing_symbol)                
#         print('for', word_with_missing_symbol, ':', )
        if not all_suggested:
            predicted_symbols.append(space_symbol)
            continue
        suggested = None
        for suggestion in all_suggested:
            if len(word_with_missing_symbol) == len(suggestion) and\
            suggestion.startswith(word_with_missing_symbol[0:index_of_missing]) and\
            suggestion.endswith(word_with_missing_symbol[index_of_missing+1:]):
                suggested = suggestion
                break
        if suggested is None:
            print('Failed at', index+1)
            print('Word:', word_with_missing_symbol)
            print('Not stripped:', line[index_of_prev_space:index_of_next_space])
            print('Suggested:', all_suggested)
            print('-'*50)
            suggested = all_suggested[0]
#         print('at', index, ':', suggested, '| index:', index_of_missing-1)
        if index_of_missing >= len(suggested):
            predicted_symbols.append(space_symbol)
        else:
            suggested_symbol = suggested[index_of_missing]
            if suggested_symbol == missing_symbol:
                predicted_symbols.append(space_symbol)
            else:
                predicted_symbols.append(suggested_symbol)

Failed at 43
Word: Pottgeite■
Not stripped:  Pottgeite■
Suggested: ['Poltergeist']
--------------------------------------------------
Failed at 63
Word: ■inganna
Not stripped:  ■inganna
Suggested: ['Nanning', 'Inanna', 'angina']
--------------------------------------------------
Failed at 107
Word: Ryter's■body
Not stripped:  Ryter's■body
Suggested: ['Afterbody']
--------------------------------------------------
Failed at 142
Word: Slove■ski
Not stripped:  Slove■ski
Suggested: ['Slovenia']
--------------------------------------------------
Failed at 145
Word: Do■inico
Not stripped:  Do■inico,
Suggested: ['Dominic']
--------------------------------------------------
Failed at 164
Word: fu■fillment
Not stripped:  fu■fillment.
Suggested: ['fulfilment']
--------------------------------------------------
Failed at 172
Word: F■nwick
Not stripped:  F■nwick.
Suggested: ['Pickwick']
--------------------------------------------------
Failed at 203
Word: but■multicolored
Not stripped:  but■multi

Failed at 1850
Word: offense■was
Not stripped:  offense■was
Suggested: ['offensiveness']
--------------------------------------------------
Failed at 1897
Word: Lawto■
Not stripped:  Lawto■
Suggested: ['Latoya']
--------------------------------------------------
Failed at 1927
Word: Da■getty
Not stripped:  Da■getty
Suggested: ['Getty']
--------------------------------------------------
Failed at 1941
Word: Fren■lyer
Not stripped:  Fren■lyer
Suggested: ['Freshener']
--------------------------------------------------
Failed at 2082
Word: Nebu-hin-Ab■noz
Not stripped: 
Suggested: ['Neb-hin-Ab■noz', 'Nebr-hin-Ab■noz', 'Nebo-hin-Ab■noz', 'Zebu-hin-Ab■noz', 'Cebu-hin-Ab■noz', 'Ne bu-hin-Ab■noz', 'Ne-bu-hin-Ab■noz', 'Neb u-hin-Ab■noz', 'Nebs-hin-Ab■noz', 'Nebuchadnezzar']
--------------------------------------------------
Failed at 2125
Word: Ameri■
Not stripped:  Ameri■
Suggested: ['Merriam']
--------------------------------------------------
Failed at 2139
Word: Coru-hin-Irigo■
Not stripped

Failed at 3853
Word: cadet■maneuvers
Not stripped:  cadet■maneuvers.
Suggested: ['cabinetmakers']
--------------------------------------------------
Failed at 3908
Word: Mocquard■
Not stripped:  Mocquard■'
Suggested: ['Jacquard']
--------------------------------------------------
Failed at 3977
Word: Zi■f-Davis
Not stripped:  Zi■f-Davis
Suggested: ['Ziti-Davis', 'Affidavits']
--------------------------------------------------
Failed at 4001
Word: Te■emsi
Not stripped:  Te■emsi.
Suggested: ['Telesis']
--------------------------------------------------
Failed at 4018
Word: Ander■
Not stripped:  Ander■'
Suggested: ['Dander']
--------------------------------------------------
Failed at 4036
Word: U.S.■copyright
Not stripped:  U.S.■copyright
Suggested: ['U. S.■copyright', 'Copyright']
--------------------------------------------------
Failed at 4038
Word: Yes,■
Not stripped: 
Suggested: ['Yes']
--------------------------------------------------
Failed at 4081
Word: ■troid
Not stripped:  ■tr

Failed at 6422
Word: Keat■ng
Not stripped:  Keat■ng.
Suggested: ['Keaton']
--------------------------------------------------
Failed at 6553
Word: Yessir.■
Not stripped: 
Suggested: ['Yessing']
--------------------------------------------------
Failed at 6595
Word: Jonner■
Not stripped:  "Jonner■
Suggested: ['Nonjoiner']
--------------------------------------------------
Failed at 6598
Word: ■anem
Not stripped:  ■anem?
Suggested: ['Neman']
--------------------------------------------------
Failed at 6615
Word: replied■Zaidie
Not stripped:  replied■Zaidie;
Suggested: ['replicability']
--------------------------------------------------
Failed at 6637
Word: Beckwith'■
Not stripped:  Beckwith'■
Suggested: ['Bewitchment']
--------------------------------------------------
Failed at 6663
Word: Sathe■i
Not stripped:  Sathe■i.
Suggested: ['Sheathe']
--------------------------------------------------
Failed at 6683
Word: ■acCulloch
Not stripped:  ■acCulloch
Suggested: ['McCullough']
-----------

Failed at 8486
Word: the■mesmerist's
Not stripped:  the■mesmerist's
Suggested: ["mesmeriser's"]
--------------------------------------------------
Failed at 8554
Word: ■ournay's
Not stripped:  ■ournay's
Suggested: ["journal's"]
--------------------------------------------------
Failed at 8577
Word: C■rmody
Not stripped:  C■rmody
Suggested: ['Corrody']
--------------------------------------------------
Failed at 8588
Word: Antin■a's
Not stripped:  Antin■a's
Suggested: ["Antigua's"]
--------------------------------------------------
Failed at 8590
Word: ■ortha
Not stripped:  ■ortha.
Suggested: ['Haworth']
--------------------------------------------------
Failed at 8599
Word: Hu■bolt
Not stripped:  Hu■bolt
Suggested: ['Humboldt']
--------------------------------------------------
Failed at 8628
Word: set■Ruhl
Not stripped:  set■Ruhl
Suggested: ['ruthless']
--------------------------------------------------
Failed at 8636
Word: Ken■arobi
Not stripped:  Ken■arobi
Suggested: ['Anaerobic']
-

...

Failed at 26746
Word: Tremon■'s
Not stripped:  Tremon■'s
Suggested: ["Tremor's"]
--------------------------------------------------
Failed at 26754
Word: hagiolo■ists
Not stripped:  hagiolo■ists
Suggested: ['histologists']
--------------------------------------------------
Failed at 26819
Word: Q■arante
Not stripped:  Q■arante_
Suggested: ['Guarantee']
--------------------------------------------------
Failed at 26865
Word: kil■meters
Not stripped:  kil■meters.
Suggested: ['milometers']
--------------------------------------------------
Failed at 26889
Word: Sigyn.■"When
Not stripped:  Sigyn.■"When
Suggested: ['Sightseeing']
--------------------------------------------------
Failed at 26893
Word: Morhan■e
Not stripped:  Morhan■e,
Suggested: ['Morbihan']
--------------------------------------------------
Failed at 26971
Word: P■aneteers
Not stripped:  P■aneteers
Suggested: ['Sonneteers']
--------------------------------------------------
Failed at 26977
Word: A■tro
Not stripped:  A■tro,

Failed at 28782
Word: why■Bemmon's
Not stripped:  why■Bemmon's
Suggested: ["persimmon's"]
--------------------------------------------------
Failed at 28802
Word: e■tra
Not stripped:  e■tra
Suggested: ['tetra']
--------------------------------------------------
Failed at 28904
Word: Maillef■u
Not stripped:  Maillef■u,
Suggested: ['Manille']
--------------------------------------------------
Failed at 28938
Word: ■orvin
Not stripped:  ■orvin
Suggested: ['corvine']
--------------------------------------------------
Failed at 28974
Word: Morhan■e
Not stripped:  Morhan■e.
Suggested: ['Morbihan']
--------------------------------------------------
Failed at 28989
Word: M■zechazz
Not stripped: 
Suggested: ['Mechanize']
--------------------------------------------------
Failed at 29013
Word: a■well-traveled
Not stripped:  a■well-traveled
Suggested: ['Falwell-traveled', 'travelled']
--------------------------------------------------
Failed at 29018
Word: #1■
Not stripped:  #1■
Suggested: ['1']


Failed at 30746
Word: L■wton
Not stripped:  L■wton
Suggested: ['Newton']
--------------------------------------------------
Failed at 30789
Word: Tifinar■writing
Not stripped:  Tifinar■writing
Suggested: ['Scriptwriting']
--------------------------------------------------
Failed at 30796
Word: K■fner
Not stripped:  K■fner,
Suggested: ['Hefner']
--------------------------------------------------
Failed at 30815
Word: Gaddon■
Not stripped: 
Suggested: ['Abaddon']
--------------------------------------------------
Failed at 30859
Word: Halger■en
Not stripped:  Halger■en
Suggested: ['Challenger']
--------------------------------------------------
Failed at 30896
Word: (_C■tizen
Not stripped: 
Suggested: ['citizen']
--------------------------------------------------
Failed at 30900
Word: S■renians
Not stripped:  S■renians,
Suggested: ['Armenians']
--------------------------------------------------
Failed at 31024
Word: crystopl■stic
Not stripped:  crystopl■stic
Suggested: ['cryptoclastic']


Failed at 32725
Word: Astro?"■asked
Not stripped:  Astro?"■asked
Suggested: ['Asterisked']
--------------------------------------------------
Failed at 32736
Word: B■rrent's
Not stripped:  B■rrent's
Suggested: ["Brent's"]
--------------------------------------------------
Failed at 32769
Word: Terr■ns
Not stripped:  Terr■ns,
Suggested: ['Terns']
--------------------------------------------------
Failed at 32776
Word: daren't■even
Not stripped:  daren't■even
Suggested: ['preventative']
--------------------------------------------------
Failed at 32873
Word: specialty■
Not stripped:  specialty■
Suggested: ['speciality']
--------------------------------------------------
Failed at 32908
Word: aiutemi■
Not stripped:  aiutemi■_
Suggested: ['Paiute']
--------------------------------------------------
Failed at 32947
Word: h■bla
Not stripped:  h■bla.
Suggested: ['blah']
--------------------------------------------------
Failed at 32963
Word: Senor_■
Not stripped: 
Suggested: ['Senor']
-------

Failed at 34621
Word: Si■yn
Not stripped:  Si■yn
Suggested: ['Sicyon']
--------------------------------------------------
Failed at 34663
Word: Marspor■
Not stripped:  Marspor■,
Suggested: ['Macrospore']
--------------------------------------------------
Failed at 34683
Word: Pl■neteers
Not stripped:  Pl■neteers
Suggested: ['Sonneteers']
--------------------------------------------------
Failed at 34727
Word: VIT■E'S
Not stripped:  VIT■E'S
Suggested: ["VITRINE'S"]
--------------------------------------------------
Failed at 34776
Word: ■ilbun
Not stripped:  ■ilbun
Suggested: ['Wilburn']
--------------------------------------------------
Failed at 34798
Word: Tr■mont
Not stripped:  Tr■mont
Suggested: ['Tramontane']
--------------------------------------------------
Failed at 34865
Word: Myla's■house
Not stripped:  Myla's■house,
Suggested: ['Amylaceous']
--------------------------------------------------
Failed at 34870
Word: millo■
Not stripped:  millo■
Suggested: ['Milo', 'milo']
-----

In [127]:
predictions = pd.DataFrame({'id': indices, 'y': predicted_symbols})
predictions.to_csv('symbol_prediction_subm_2.csv', sep=',')

In [110]:
separators = [' ', '!', '?', '"', '\'', '.', ',', ':', ';']
word_with_missing_symbol = 'rig■t,'.strip(''.join(separators))
all_suggested = spellchecker.suggest(word_with_missing_symbol)
print(all_suggested)
print(len(word_with_missing_symbol) == len(all_suggested[0]))
index_of_missing = word_with_missing_symbol.find(missing_symbol)
print(index_of_missing)
print(word_with_missing_symbol[0:index_of_missing], all_suggested[0].startswith(word_with_missing_symbol[0:index_of_missing]))
print(word_with_missing_symbol[index_of_missing+1:], all_suggested[0].endswith(word_with_missing_symbol[index_of_missing+1:]))

['right']
True
3
rig True
t True


In [117]:
line = test_df['text'][74]
index_of_missing = line.find(missing_symbol)
index_of_prev_space = line.rfind(space_symbol, 0, index_of_missing)
index_of_next_space = line.find(space_symbol, index_of_missing+1)
prev_word = line[index_of_prev_space:index_of_missing].strip()
next_word = line[index_of_missing+1:index_of_next_space+1].strip()